In [28]:
import pandas as pd
import numpy as np

In [14]:
file_path = '../data/ESP_PUBLIC.IDENTITE_ARBRE.csv'
df = pd.read_csv(file_path, sep=',', header = 0, index_col=False)

In [15]:
df

,ELEM_POINT_ID,CODE,NOM,GENRE,GENRE_DESC,CATEGORIE,CATEGORIE_DESC,SOUS_CATEGORIE,SOUS_CATEGORIE_DESC,CODE_PARENT,...,COURRIER,IDENTIFIANTPLU,TYPEIMPLANTATIONPLU,INTITULEPROTECTIONPLU,ANNEEABATTAGE,ESSOUCHEMENT,DIAMETREARBRE,CAUSEABATTAGE,COLLECTIVITE,GeoJSON
0,23881,ESP29285,ESP29285,VEG,VEGETATION,ESP01,Arbre,ESP174,Arbre d'espaces ouverts,ESP639,...,NaN,170.0,Alignement,L 123-1.7,NaN,NaN,NaN,NaN,Ville de Grenoble,"{""type"":""Point"",""coordinates"":[5.7436272177661..."
1,23886,ESP29286,ESP29286,VEG,VEGETATION,ESP01,Arbre,ESP187,Bois semi naturel,ESP53,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Ville de Grenoble,"{""type"":""Point"",""coordinates"":[5.7448701441213..."
2,23891,ESP29287,ESP29287,VEG,VEGETATION,ESP01,Arbre,ESP174,Arbre d'espaces ouverts,ESP1058,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Grenoble Alpes Métropole,"{""type"":""Point"",""coordinates"":[5.7422258900484..."
3,23895,ESP29288,ESP29288,VEG,VEGETATION,ESP01,Arbre,ESP151,Arbre de voirie,ESP616,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Ville de Grenoble,"{""type"":""Point"",""coordinates"":[5.7447571861843..."
4,12182,ESP29289,ESP29289,VEG,VEGETATION,ESP01,Arbre,ESP174,Arbre d'espaces ouverts,ESP1213,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Ville de Grenoble,"{""type"":""Point"",""coordinates"":[5.7366655831041..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31614,30028,ESP31275,ESP31275,VEG,VEGETATION,ESP01,Arbre,ESP174,Arbre d'espaces ouverts,ESP42,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Ville de Grenoble,"{""type"":""Point"",""coordinates"":[5.7243716154461..."
31615,30029,ESP31276,ESP31276,VEG,VEGETATION,ESP01,Arbre,ESP174,Arbre d'espaces ouverts,ESP42,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Ville de Grenoble,"{""type"":""Point"",""coordinates"":[5.7245238150976..."
31616,30030,ESP31277,ESP31277,VEG,VEGETATION,ESP01,Arbre,ESP174,Arbre d'espaces ouverts,ESP42,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Ville de Grenoble,"{""type"":""Point"",""coordinates"":[5.7246681137245..."
31617,30031,ESP31278,ESP31278,VEG,VEGETATION,ESP01,Arbre,ESP174,Arbre d'espaces ouverts,ESP42,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Ville de Grenoble,"{""type"":""Point"",""coordinates"":[5.7249966111508..."


# Tuto Pandas - Cheatsheet

#### 2. How to create a data frame using a dictionary of pre-existing columns or NumPy 2D arrays?

In [32]:
# c1, c2, c3, c4 are column names. 
d_dic ={
    "first_col_name":[1, 2, 4, 9],
    "second_col_names":["oui", "bar", "blabla", "foo"],
    "3rd_col_name":[41, 55, 12, 12]
} 
df = pd.DataFrame(data = d_dic)
df

,first_col_name,second_col_names,3rd_col_name
0,1,oui,41
1,2,bar,55
2,4,blabla,12
3,9,foo,12


#### 3. How to visualize the top and bottom x values in a data frame?